In [1]:
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
from syft.federated.floptimizer import Optims
from torch import nn, optim


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/guoxinli/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


## baisc api

In [2]:
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [7]:
bob = sy.VirtualWorker(hook, id="bob")
x = torch.tensor([1.123123,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [8]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [10]:
x_ptr.fix_precision()

AttributeError: 'Tensor' object has no attribute 'child'

In [6]:
bob._objects

{97871883798: tensor([1, 2, 3, 4, 5]), 34506428097: tensor([1, 1, 1, 1, 1])}

In [7]:
z = x_ptr + x_ptr

In [8]:
z

(Wrapper)>[PointerTensor | me:33813020194 -> bob:48798582415]

In [9]:
bob._objects

{97871883798: tensor([1, 2, 3, 4, 5]),
 34506428097: tensor([1, 1, 1, 1, 1]),
 48798582415: tensor([ 2,  4,  6,  8, 10])}

In [11]:
print(x_ptr.location)
print(x_ptr.id_at_location)
print(x_ptr.id)
print(x_ptr.owner)

<VirtualWorker id:bob #objects:3>
97871883798
91770630647
<VirtualWorker id:me #objects:0>


In [13]:
sy.local_worker

<VirtualWorker id:me #objects:0>

In [15]:
x_ptr.get()

tensor([ 2,  4,  6,  8, 10])

## Fed learning

In [9]:
import torch
from torch import nn
from torch import optim

In [3]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [5]:
train()

tensor(1.7342)
tensor(0.3538)
tensor(0.1971)
tensor(0.1451)
tensor(0.1104)
tensor(0.0844)
tensor(0.0645)
tensor(0.0493)
tensor(0.0377)
tensor(0.0288)
tensor(0.0220)
tensor(0.0168)
tensor(0.0129)
tensor(0.0098)
tensor(0.0075)
tensor(0.0058)
tensor(0.0044)
tensor(0.0034)
tensor(0.0026)
tensor(0.0020)


In [32]:
hook = sy.TorchHook(torch)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


In [4]:
from syft.workers.websocket_client import WebsocketClientWorker

bob = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='bob', 
                            port=8778)
alice = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='alice', 
                            port=8777)

secure_worker = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='james', 
                            port=8779)

{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8778'}
{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8777'}
{'max_size': None, 'timeout': 60, 'url': 'ws://localhost:8779'}


In [5]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

/Users/guoxinli/.pyenv/versions/3.7.0/lib/python3.7/site-packages/torch/tensor.py:746: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  warnings.warn("The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad "


In [6]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [7]:
def train():
    # Training Logic
    for iter in range(2):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
#             print(data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()
            for name, param in model.named_parameters():
                print(name, param)
            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [8]:
model

Linear(in_features=2, out_features=1, bias=True)

In [9]:
train()

Websocket connection closed (worker: bob)
Created new websocket connection


RuntimeError: Websocket connection closed and creation of new connection failed.

## Trusted Aggregator

In [4]:
hook = sy.TorchHook(torch)
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# Iniitalize A Toy Model
model = nn.Linear(2,1)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [5]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(1.1187) Alice:tensor(0.6470)
Bob:tensor(0.2573) Alice:tensor(0.0628)
Bob:tensor(0.0614) Alice:tensor(0.0486)
Bob:tensor(0.0164) Alice:tensor(0.0404)
Bob:tensor(0.0059) Alice:tensor(0.0336)
Bob:tensor(0.0032) Alice:tensor(0.0280)
Bob:tensor(0.0023) Alice:tensor(0.0233)
Bob:tensor(0.0019) Alice:tensor(0.0194)
Bob:tensor(0.0016) Alice:tensor(0.0161)
Bob:tensor(0.0013) Alice:tensor(0.0134)


In [7]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))


Bob:tensor(0.0097) Alice:tensor(0.0105)
Bob:tensor(0.0104) Alice:tensor(0.0052)
Bob:tensor(0.0099) Alice:tensor(0.0027)
Bob:tensor(0.0089) Alice:tensor(0.0015)
Bob:tensor(0.0076) Alice:tensor(0.0008)
Bob:tensor(0.0063) Alice:tensor(0.0005)
Bob:tensor(0.0051) Alice:tensor(0.0003)
Bob:tensor(0.0041) Alice:tensor(0.0002)
Bob:tensor(0.0033) Alice:tensor(0.0001)
Bob:tensor(0.0026) Alice:tensor(8.3656e-05)


In [8]:

preds = model(data)
loss = ((preds - target) ** 2).sum()

print(preds)
print(target)
print(loss.data)

tensor([[0.1451],
        [0.1203],
        [0.8480],
        [0.8232]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0899)


In [9]:
alices_model.location

<VirtualWorker id:secure_worker #objects:4>

In [47]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

In [54]:
y = p2p2x + p2p2x

In [55]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [58]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [61]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {37385115291: tensor([[0.2698],
        [0.7894]], grad_fn=<AddmmBackward>), 79445952521: tensor([[0.0000, 0.1579]]), 77015351021: tensor([[0.0000, 0.0025]]), 16937095376: tensor([0.2118]), 67549564842: tensor([0.0045]), 75554866810: tensor([[0., 0.],
        [0., 1.]], requires_grad=True), 50248922242: tensor([[0.],
        [0.]], requires_grad=True), 92972669725: tensor([[0.0000, 0.1493]]), 23270153660: tensor([[0.0000, 0.0008]]), 64117208361: tensor([0.4620]), 79364950590: tensor([0.0073]), 40429147059: tensor([ 2,  4,  6,  8, 10]), 44164372654: tensor([1, 2, 3, 4, 5])}
alice: {42341012943: tensor([[1., 0.],
        [1., 1.]], requires_grad=True), 8908801304: tensor([[1.],
        [1.]], requires_grad=True), 64893623566: tensor([[-0.0467, -0.1392]]), 81934718163: tensor([[0.0005, 0.0008]]), 38343751621: tensor([-0.0467]), 91910903521: tensor([0.0005])}


In [65]:
x = x.move(bob)

In [67]:

print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {37385115291: tensor([[0.2698],
        [0.7894]], grad_fn=<AddmmBackward>), 79445952521: tensor([[0.0000, 0.1579]]), 77015351021: tensor([[0.0000, 0.0025]]), 16937095376: tensor([0.2118]), 67549564842: tensor([0.0045]), 75554866810: tensor([[0., 0.],
        [0., 1.]], requires_grad=True), 50248922242: tensor([[0.],
        [0.]], requires_grad=True), 92972669725: tensor([[0.0000, 0.1493]]), 23270153660: tensor([[0.0000, 0.0008]]), 64117208361: tensor([0.4620]), 79364950590: tensor([0.0073]), 40429147059: tensor([ 2,  4,  6,  8, 10]), 44164372654: tensor([1, 2, 3, 4, 5])}
alice: {42341012943: tensor([[1., 0.],
        [1., 1.]], requires_grad=True), 8908801304: tensor([[1.],
        [1.]], requires_grad=True), 64893623566: tensor([[-0.0467, -0.1392]]), 81934718163: tensor([[0.0005, 0.0008]]), 38343751621: tensor([-0.0467]), 91910903521: tensor([0.0005])}


In [66]:
x

(Wrapper)>[PointerTensor | me:2048474029 -> bob:44164372654]

0